In [111]:
import pandas as pd

import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem import LancasterStemmer 

nltk.download("stopwords")

import string
import re

import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import SGDClassifier

import numpy as np

from scipy import sparse
import datetime

import s3fs

# import lch_proprietary.ml_modeling as ml 

from sklearn.feature_extraction.text import TfidfVectorizer

import io

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
test = pd.read_csv("s3://advancedml-koch-mathur-hinkson/test.csv")

In [3]:
train = pd.read_csv("s3://advancedml-koch-mathur-hinkson/train.csv")

Split train.csv into train (80%) and validation sets (20%).

In [6]:
validation_set = train[:360000]

In [8]:
train_set = train[360000:]

To speed up process, create small samples of the train_set on which to run NB.  Ensure that samples are iid by replacing after each draw.

In [26]:
train_sample1 = train_set.sample(frac=0.5, replace=True, random_state=1)

In [29]:
train_sample1.shape

(722437, 45)

In [27]:
ls = LancasterStemmer()
ps = PorterStemmer() 

sw = set(stopwords.words('english'))
sw.add('')

def clean_text(text, stemming=None, remove_sw = True):

    t = text.replace("-", " ").split(" ")
    
    t = [w.lower() for w in t]
    
    if remove_sw == True:
        t = [w for w in t if w not in sw]
    
    if stemming == None:
        pass;
    elif stemming == "Porter":
        t = [ps.stem(w) for w in t]
    elif stemming == "Lancaster":
        t = [ls.stem(w) for w in t]
    else:
        print("Please enter a valid stemming type")
        
    t = [w.strip(string.punctuation) for w in t]


    return ' '.join(t)

In [28]:
def add_text_cleaning_cols(df):
    print(datetime.datetime.now())
    
    df['split'] = df["comment_text"].apply(lambda x: x.split(" "))
    df['cleaned_w_stopwords'] = df["comment_text"].apply(clean_text,args=(None,False),)

    print(datetime.datetime.now())
    df['cleaned_no_stem'] = df["comment_text"].apply(clean_text,)
    df['cleaned_porter'] = df["comment_text"].apply(clean_text,args=("Porter",),)
    df['cleaned_lancaster'] = df["comment_text"].apply(clean_text,args=("Lancaster",),)

    print(datetime.datetime.now())

    df['perc_upper'] = df["comment_text"].apply(lambda x: round((len(re.findall(r'[A-Z]',x)) / len(x)), 3))

    df['num_exclam'] = df["comment_text"].apply(lambda x:(len(re.findall(r'!',x))))
    
    df['num_words'] = df["split"].apply(lambda x: len(x))
    print(datetime.datetime.now())

    df['perc_stopwords'] = round((df.num_words - df['cleaned_no_stem'].apply(lambda x: len(x)))/df.num_words,3) 
    
    df['num_upper_words'] = df["split"].apply(lambda x: sum(map(str.isupper, x)) 
)
    print(datetime.datetime.now())

    
    

In [30]:
add_text_cleaning_cols(train_sample1)

2019-05-25 21:19:49.786419
2019-05-25 21:20:10.558744
2019-05-25 21:29:42.878572
2019-05-25 21:29:47.732161
2019-05-25 21:29:52.538530


In [31]:
train_sample1.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count', 'split',
       'cleaned_w_stopwords', 'cleaned_no_stem', 'cleaned_porter',
       'cleaned_lancaster', 'perc_upper', 'num_exclam', 'num_words',
       'perc_stopwords', 'num_upper_words'],
      dtype='object')

Pickle the dataset and send to s3 bucket:

In [115]:

pickle_buffer = io.BytesIO()
s3_resource = boto3.resource('s3')
bucket='advancedml-koch-mathur-hinkson'
key='preprocessed_train_sample_50pct.pkl'

# test.to_pickle(pickle_buffer)
# s3_resource.Object(bucket, 'full_preprocessed_test.pkl').put(Body=pickle_buffer.getvalue())

test.to_pickle(key)
s3_resource.Object(bucket,key).put(Body=open(key, 'rb'))


{'ResponseMetadata': {'RequestId': '3D08FE279E1DFF4D',
  'HostId': 'JlkdfeNjOxKkim8osIrvDNH4FosgJifyZrAjvnMeTVsuJEjqJVg6a40it0wYV65bbe00zWu6T5I=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'JlkdfeNjOxKkim8osIrvDNH4FosgJifyZrAjvnMeTVsuJEjqJVg6a40it0wYV65bbe00zWu6T5I=',
   'x-amz-request-id': '3D08FE279E1DFF4D',
   'date': 'Sat, 25 May 2019 23:05:55 GMT',
   'etag': '"74e5f9fe7a29990cbb2b86b7cc4296e0"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"74e5f9fe7a29990cbb2b86b7cc4296e0"'}

In [32]:
addtl_cols = ['perc_upper', 'num_exclam', 'num_words', 'perc_stopwords', 'num_upper_words']
#addtl_cols = ['perc_upper']

In [40]:
train_sample1.shape

(722437, 55)

In [39]:
train_sample1.head(5)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,split,cleaned_w_stopwords,cleaned_no_stem,cleaned_porter,cleaned_lancaster,perc_upper,num_exclam,num_words,perc_stopwords,num_upper_words
488037,842255,0.0,"What, you don't think the earth is at least 8,...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,"[What,, you, don't, think, the, earth, is, at,...","what you don't think the earth is at least 8,0...","what think earth least 8,000 years old","what think earth least 8,000 year old","what think ear least 8,000 year old",0.017,0,12,-2.167,0
851755,5162644,0.0,"IMO one reason is the ""degree of foresight"". ...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,"[IMO, one, reason, is, the, ""degree, of, fores...",imo one reason is the degree of foresight tho...,imo one reason degree foresight less foresight...,imo one reason degre foresight less foresight ...,imo on reason degree foresight less foresight ...,0.019,0,53,-1.660,1
830924,5137657,0.0,""" Oh- and throw in some more ""racist"" and ""mys...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,"["", Oh-, and, throw, in, some, more, ""racist"",...",oh and throw in some more racist and mysogen...,oh throw racist mysogeny name calls supporter...,oh throw racist mysogeny name call support wo...,oh throw racist mysogeny nam cal support wond...,0.013,0,36,-3.306,0
851263,5162070,0.0,If anyone understands the subtle intricacies o...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,"[If, anyone, understands, the, subtle, intrica...",if anyone understands the subtle intricacies o...,anyone understands subtle intricacies saying o...,anyon understand subtl intricaci say one thing...,anyon understand subtl int say on thing vot an...,0.015,0,34,-3.441,0
1196489,5578177,0.0,"""klu klux klan""?",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"[""klu, klux, klan""?]",klu klux klan,klu klux klan,klu klux klan,klu klux klan,0.000,0,3,-3.333,0


Due to memory issues, we needed to use a smaller training set.  We used a random iid sample of half of the train_sample1 frame to train NB model:

In [53]:
subset_train_sample1 = train_sample1.sample(frac=0.1, replace=True, random_state=1)

In [88]:
subset_train_sample1.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count', 'split',
       'cleaned_w_stopwords', 'cleaned_no_stem', 'cleaned_porter',
       'cleaned_lancaster', 'perc_upper', 'num_exclam', 'num_words',
       'perc_stopwords', 'num_upper_words'],
      dtype='object')

Because we are unable to train an NB model on categorical (text) and continuous (numerical) data at the same time, our action plan changed to running two independent models for each type of data and then running a thrid NB model on the resulting predict_proba from the other two trained models.

In [112]:
def run_model(model_df, train_perc=.80, addtl_feats =[''], model_type = "Multi", 
              num_iter = 10, should_print=False, see_inside=False, comments="comment_text"):
    
    train_start = 0
    train_end = round(model_df.shape[0]*train_perc) 

    test_start = train_end
    test_end = model_df.shape[0]
    
    X_all = model_df[comments].values
    y_all = model_df['target'].values
    
    # tokenizing text
#     count_vect = CountVectorizer()
#     X_all_counts = count_vect.fit_transform(X_all.astype('U'))
    #print(X_all_counts.shape)

    # calculating frequencies
    tfidf_vectorizer = TfidfVectorizer(use_idf=True)
    fitted_vectorizer=tfidf_vectorizer.fit(model_df[comments])
    X_all_tfidf =  fitted_vectorizer.transform(model_df[comments])


    print(X_all_tfidf.shape)
    
    if addtl_feats != ['']: # combine non-text and text features if necessary
        print("here")
#         others_all = model_df[addtl_feats].values.reshape(-1,1)

        others_all = model_df[addtl_feats].values.reshape(-1,len(addtl_feats))
        #print(others_all)
        newfeatures_all = sparse.hstack((X_all_tfidf, others_all.astype(float))).tocsr()
    else:
        newfeatures_all = X_all_tfidf
    
    
    X_train = newfeatures_all[train_start:train_end]
    y_train = model_df[train_start:train_end]['target'].values
    y_train=y_train.astype('int')

    X_test = newfeatures_all[test_start:test_end]
    y_test = model_df[test_start:test_end]['target'].values
    
    
    if model_type == 'Multi':
        clf = MultinomialNB().fit(X_train, y_train)
    if model_type == "Gauss":
        clf = GaussianNB().fit(X_train, y_train) 
   
    predicted = clf.predict(X_test)
    accuracy = np.mean(predicted == y_test)
    

#     y_scores_sorted, y_true_sorted = joint_sort_descending(np.array(y_scores), np.array(y_true))
#     precision = precision_score(y_true_sorted, preds)


    if should_print == True:

        print("The accuracy on the test set is {}%.".format(round(accuracy*100,2)))    
    
    if see_inside == True:
        return clf, accuracy, X_all_counts, X_all_tfidf
    else:
        return clf, accuracy


In [113]:
clf1, accuracy = run_model(subset_train_sample1, should_print=True)

(72244, 65038)
The accuracy on the test set is 70.18%.


In [114]:
clf2, accuracy = run_model(subset_train_sample1, should_print=True, addtl_feats =['insult', 'perc_upper'])

(72244, 65038)
here
The accuracy on the test set is 70.18%.


In [92]:
for text in ['cleaned_w_stopwords', 'cleaned_no_stem', 'cleaned_porter','cleaned_lancaster']:
    print(text)
    run_model(subset_train_sample1, comments=text, should_print=True)

cleaned_w_stopwords
(72244, 64969)
The accuracy on the test set is 70.18%.
cleaned_no_stem
(72244, 64967)
The accuracy on the test set is 70.18%.
cleaned_porter
(72244, 61279)
The accuracy on the test set is 70.18%.
cleaned_lancaster
(72244, 58684)
The accuracy on the test set is 70.18%.


In [ ]:
    `